# Data Mining
## Semester Project
### Supervisors: Dr. Sibt-ul-Hussain and Atique-ur-Rehman
#### By: Zeeshan Ali, i14-1623, A
###### Cleaning of Test Data -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
%pylab inline
import scipy.stats
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import os


plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
print "Matplotlib version : ", matplotlib.__version__
print "Pandas version : ", pd.__version__
print "Numpy version : ", np.__version__
print "Scipy version : ", scipy.__version__
print "Seaborn version : ", sns.__version__
print "Sklearn version : ", sk.__version__
print "Tensorflow version : ", tf.__version__

Matplotlib version :  2.0.0
Pandas version :  0.19.2
Numpy version :  1.11.3
Scipy version :  0.18.1
Seaborn version :  0.7.1
Sklearn version :  0.18.1
Tensorflow version :  1.1.0


# Region Information Table
It shows the information about the regions to be evaluated in the contest.  You need to do the prediction given the regions from the Region Definition Table.  In the submission of the results, you need to map the region hash value to region mapped ID.

In [3]:
cluster_map_dir = './test_set/cluster_map/'
cluster_map_files = [f for f in os.listdir(cluster_map_dir) if not f.startswith('.')]
print 'Number of cluster_map files =', len(cluster_map_files)
cluster_map_data = []
for File in cluster_map_files:
    cluster_map_data.append(pd.read_table(cluster_map_dir+File, error_bad_lines=False, index_col=None, header=-1))
cluster_map_data = pd.concat(cluster_map_data, ignore_index=True)
cluster_map_data.columns = ['region_hash', 'region_id']
cluster_map_data.tail()

Number of cluster_map files = 1


,region_hash,region_id
61,a735449c5c09df639c35a7d61fad3ee5,62
62,0a5fef95db34383403d11cb6af937309,63
63,bf44d327f0232325c6d5280926d7b37d,64
64,825a21aa308dea206adb49c4b77c7805,65
65,1ecbb52d73c522f184a6fc53128b1ea1,66


# Order Information Table
It  shows  the  basic  information  of  an  order,  including  the  passenger  and  the  driver  (if driver id =NULL, it means the order was not answered by any driver), place of origin, destination, price and time.  The fields order id, driver id, passenger id, start hash, and dest hash are made not sensitive.

In [4]:
order_data_dir = './test_set/order_data/'
order_data_files = [f for f in os.listdir(order_data_dir) if not f.startswith('.')]
print 'Number of order_data files =', len(order_data_files)
order_data = []
for File in order_data_files:
    order_data.append(pd.read_csv(order_data_dir+File, error_bad_lines=False, index_col=None, header=-1))
order_data = pd.concat(order_data, ignore_index=True)
order_data.columns = ['order_id', 'passenger_id', 'start_region_hash', 'dest_region_hash', 'Time']
order_data.tail()

Number of order_data files = 5


,order_id,passenger_id,start_region_hash,dest_region_hash,Time
555118,7f7967740241da1fd4b63c1cb62cef11,3684566f98450b4366fd2eaeab775949,90c5a34f06ac86aee0fd70e2adce7d8a,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-23 13:15:33
555119,545e612cb0ab33eb01a30d25788d0b79,73fd46dfdada59a9b3200c0c23d7aab6,1afd7afbc81ecc1b13886a569d869e8a,1afd7afbc81ecc1b13886a569d869e8a,2016-01-23 23:05:59
555120,ee69090096d5842cc83631a17bb5ae8d,2768592d874e983d0dc5c22ee45876ef,91690261186ae5bee8f83808ea1e4a01,2407d482f0ffa22a947068f2551fe62c,2016-01-23 17:27:23
555121,89f2989695a2efde6ae9dcc2813d7c36,d61f4f790167bf62cfa148618480f28c,91690261186ae5bee8f83808ea1e4a01,1afd7afbc81ecc1b13886a569d869e8a,2016-01-23 21:28:30
555122,5d3b53d41b5ebc08863208e7c1f2fd63,ac24d21eb96fa110c90d484a84afd1cc,d4ec2125aff74eded207d2d915ef682f,d4ec2125aff74eded207d2d915ef682f,2016-01-23 11:03:07


In [5]:
print len(order_data.start_region_hash.unique()), len(order_data.dest_region_hash.unique())

66 407


# POI Information Table
The  POI  Information  Table  shows  the  attributes  of  a  region,  such  as  the  number  of  different  facilities.

**For example:**

2#1:22 means in this region, there are 22 facilities of the facility class 2#1.

2#1 means the first level class is 2 and the second level is 1, such as entertainment#theater, shopping#home appliance, sports#others.

Each class and its number is separated by a colon ':'

In [6]:
poi_data_dir = './test_set/poi_data/'
poi_data_files = [f for f in os.listdir(poi_data_dir) if not f.startswith('.')]
print 'Number of poi_data files =', len(poi_data_files)
poi_data = []
columns = ['region_hash']
columns += [str(i+1) for i in range(152)]
for File in poi_data_files:
    poi_data.append(pd.read_table(poi_data_dir+File, error_bad_lines=False, index_col=None, header=-1, names=columns))
poi_data = pd.concat(poi_data, ignore_index=True)
poi_data.tail()

Number of poi_data files = 1


,region_hash,1,2,3,4,5,6,7,8,9,...,143,144,145,146,147,148,149,150,151,152
61,d4ec2125aff74eded207d2d915ef682f,4#16:1577,1#10:332,24:3486,25:15272,20:197125,22:1162,23:13612,4:44903,8:3818,...,2#4:581,2#5:2988,2#6:12699,2#7:2656,2#1:581,2#2:21746,2#3:1577,21#1:83,13#2:83,NaN
62,82cc4851f9e4faa4e54309f8bb73fd7c,4#16:498,1#10:830,24:6308,25:4897,20:56440,22:2739,23:4980,4:22576,8:996,...,2#3:664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,fff4e8465d1e12621bc361276b6217cf,20#7:1494,20#5:249,20#4:1660,8#4:747,20#1:1494,16#12:332,16#10:581,20#8:3818,23#3:83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,b05379ac3f9b7d99370d443cfd5dcc28,4#16:166,1#10:332,24:1411,25:1826,20:37433,22:2324,23:2241,4:9628,5#4:83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,fc34648599753c9e74ab238e9a4a07ad,4#16:498,1#10:166,24:996,25:1411,20:32951,22:1079,23:747,4:7636,8:332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#  Weather Information Table
The Weather Information Table shows the weather info every 10 minutes each city.  The weather field gives the weather
conditions such as sunny, rainy, and snowy etc; all sensitive information has been removed.  The unit of temperature is
Celsius degree, and PM2_5 is the level of air pollutions.

In [7]:
weather_data_dir = './test_set/weather_data/'
weather_data_files = [f for f in os.listdir(weather_data_dir) if not f.startswith('.')]
print 'Number of weather_data files =', len(weather_data_files)
weather_data = []
for File in weather_data_files:
    weather_data.append(pd.read_table(weather_data_dir+File, error_bad_lines=False, index_col=None, header=-1))
weather_data = pd.concat(weather_data, ignore_index=True)
weather_data.columns = ['Time', 'Weather', 'temperature', 'PM2_5']
weather_data.tail()

Number of weather_data files = 5


,Time,Weather,temperature,PM2_5
82,2016-01-29 21:10:59,3,4.0,46
83,2016-01-29 21:21:18,3,4.0,46
84,2016-01-29 23:01:52,3,4.0,61
85,2016-01-29 23:11:19,3,4.0,61
86,2016-01-29 23:20:48,3,4.0,61


# Region-Group-Facilities Information Table

In [8]:
poi_cols = []
for r in poi_data:
    poi_cols.append(r)
poi_cols = poi_cols[1:]

facilities = []
for p in range(len(poi_data)):
    for col in poi_cols:
        string = poi_data[col][p]
        if type(string)==type('str'):
            new_row = ['', '', '']
            new_row[0] = poi_data['region_hash'][p]
            new_row[1] = string[:string.index(':')]
            new_row[2] = string[string.index(':')+1:]
            facilities.append(new_row)
        
facilities = pd.DataFrame(facilities, columns=['region_hash', 'Group', 'Facilities'])
facilities.tail()

,region_hash,Group,Facilities
6581,fc34648599753c9e74ab238e9a4a07ad,2#7,1079
6582,fc34648599753c9e74ab238e9a4a07ad,2#1,498
6583,fc34648599753c9e74ab238e9a4a07ad,2#2,4731
6584,fc34648599753c9e74ab238e9a4a07ad,21#2,83
6585,fc34648599753c9e74ab238e9a4a07ad,21#1,83


# Region-Weighted_Mean_Facilities Information Table

In [9]:
# weights of each class, say 1#2.
weight = defaultdict()
for g in facilities.Group:
    if g not in weight:
        weight[g] = 0
    else:
        weight[g] += 1
weight

defaultdict(None,
            {'1': 55,
             '1#1': 30,
             '1#10': 27,
             '1#11': 56,
             '1#2': 53,
             '1#3': 36,
             '1#4': 36,
             '1#5': 60,
             '1#6': 36,
             '1#7': 19,
             '1#8': 55,
             '1#9': 25,
             '10#1': 0,
             '11': 62,
             '11#1': 45,
             '11#2': 51,
             '11#3': 47,
             '11#4': 61,
             '11#5': 39,
             '11#6': 61,
             '11#7': 54,
             '11#8': 63,
             '12': 3,
             '13#1': 0,
             '13#2': 0,
             '13#3': 0,
             '13#4': 65,
             '13#5': 29,
             '13#6': 33,
             '13#7': 0,
             '13#8': 35,
             '14': 43,
             '14#1': 1,
             '14#10': 36,
             '14#2': 27,
             '14#3': 46,
             '14#4': 1,
             '14#5': 0,
             '14#6': 51,
             '14#7': 26,
        

In [10]:
weighted_sum = {}
Sum = np.sum(weight.values())    # sum of all the weights in 'weight' dictionary
for r in range(len(poi_data)):
    weighted_sum[poi_data.region_hash[r]] = 0
    for col in poi_cols:
        string = poi_data[col][r]
        if type(string) == type('str'):
            w = weight[string[:string.index(':')]]
            weighted_sum[poi_data.region_hash[r]] += w * int(string[string.index(':')+1:])
    weighted_sum[poi_data.region_hash[r]] /= Sum
weighted_sum = pd.DataFrame(weighted_sum.items(), columns=['region_hash', 'Facilities'])
weighted_sum.tail()

,region_hash,Facilities
61,1cbfbdd079ef93e74405c53fcfff8567,977
62,4b9e4cf2fbdc8281b8a1f9f12b80ce4d,253
63,82cc4851f9e4faa4e54309f8bb73fd7c,14166
64,4b7f6f4e2bf237b6cc58f57142bea5c0,987
65,2350be163432e42270d2670cb3c02f80,1209


# Supply-Demand Information Table

In [11]:
Supply_Demand_data = order_data[['start_region_hash', 'Time']]
Supply_Demand_data.tail()

,start_region_hash,Time
555118,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-23 13:15:33
555119,1afd7afbc81ecc1b13886a569d869e8a,2016-01-23 23:05:59
555120,91690261186ae5bee8f83808ea1e4a01,2016-01-23 17:27:23
555121,91690261186ae5bee8f83808ea1e4a01,2016-01-23 21:28:30
555122,d4ec2125aff74eded207d2d915ef682f,2016-01-23 11:03:07


In [12]:
print len(Supply_Demand_data['Time'].unique()), len(weather_data['Time'].unique())

76070 87


In [13]:
Supply_Demand_data = pd.merge(Supply_Demand_data, weather_data, how='left')
Supply_Demand_data.head()

,start_region_hash,Time,Weather,temperature,PM2_5
0,d4ec2125aff74eded207d2d915ef682f,2016-01-27 09:06:48,NaN,NaN,NaN
1,b05379ac3f9b7d99370d443cfd5dcc28,2016-01-27 07:18:36,NaN,NaN,NaN
2,44c097b7bd219d104050abbafe51bd49,2016-01-27 13:03:09,NaN,NaN,NaN
3,82cc4851f9e4faa4e54309f8bb73fd7c,2016-01-27 21:13:41,NaN,NaN,NaN
4,38d5ad2d22b61109fd8e7b43cd0e8901,2016-01-27 19:26:01,NaN,NaN,NaN


In [14]:
Supply_Demand_data.tail()

,start_region_hash,Time,Weather,temperature,PM2_5
555118,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-23 13:15:33,NaN,NaN,NaN
555119,1afd7afbc81ecc1b13886a569d869e8a,2016-01-23 23:05:59,NaN,NaN,NaN
555120,91690261186ae5bee8f83808ea1e4a01,2016-01-23 17:27:23,NaN,NaN,NaN
555121,91690261186ae5bee8f83808ea1e4a01,2016-01-23 21:28:30,NaN,NaN,NaN
555122,d4ec2125aff74eded207d2d915ef682f,2016-01-23 11:03:07,NaN,NaN,NaN


In [15]:
day = []
month = []
time_slot = []
for t in Supply_Demand_data['Time']:
    day.append(t[8:10])
    month.append(t[5:7])
    time_slot.append((int(t[11:13])*60*60+int(t[14:16])*60+int(t[17:19]))//600)

Supply_Demand_data = Supply_Demand_data.assign(Day = day)
Supply_Demand_data = Supply_Demand_data.assign(Month = month)
Supply_Demand_data = Supply_Demand_data.assign(Time_slot = time_slot)
Supply_Demand_data.tail()

,start_region_hash,Time,Weather,temperature,PM2_5,Day,Month,Time_slot
555118,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-23 13:15:33,NaN,NaN,NaN,23,01,79
555119,1afd7afbc81ecc1b13886a569d869e8a,2016-01-23 23:05:59,NaN,NaN,NaN,23,01,138
555120,91690261186ae5bee8f83808ea1e4a01,2016-01-23 17:27:23,NaN,NaN,NaN,23,01,104
555121,91690261186ae5bee8f83808ea1e4a01,2016-01-23 21:28:30,NaN,NaN,NaN,23,01,128
555122,d4ec2125aff74eded207d2d915ef682f,2016-01-23 11:03:07,NaN,NaN,NaN,23,01,66


In [16]:
Supply_Demand_data=Supply_Demand_data.merge(weighted_sum, left_on='start_region_hash', right_on='region_hash', how='left')
Supply_Demand_data.tail()

,start_region_hash,Time,Weather,temperature,PM2_5,Day,Month,Time_slot,region_hash,Facilities
555118,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-23 13:15:33,NaN,NaN,NaN,23,01,79,90c5a34f06ac86aee0fd70e2adce7d8a,6055
555119,1afd7afbc81ecc1b13886a569d869e8a,2016-01-23 23:05:59,NaN,NaN,NaN,23,01,138,1afd7afbc81ecc1b13886a569d869e8a,4619
555120,91690261186ae5bee8f83808ea1e4a01,2016-01-23 17:27:23,NaN,NaN,NaN,23,01,104,91690261186ae5bee8f83808ea1e4a01,2218
555121,91690261186ae5bee8f83808ea1e4a01,2016-01-23 21:28:30,NaN,NaN,NaN,23,01,128,91690261186ae5bee8f83808ea1e4a01,2218
555122,d4ec2125aff74eded207d2d915ef682f,2016-01-23 11:03:07,NaN,NaN,NaN,23,01,66,d4ec2125aff74eded207d2d915ef682f,31088


In [17]:
Supply_Demand_data = pd.merge(Supply_Demand_data, cluster_map_data, how='left')
Supply_Demand_data.tail()

,start_region_hash,Time,Weather,temperature,PM2_5,Day,Month,Time_slot,region_hash,Facilities,region_id
555118,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-23 13:15:33,NaN,NaN,NaN,23,01,79,90c5a34f06ac86aee0fd70e2adce7d8a,6055,1
555119,1afd7afbc81ecc1b13886a569d869e8a,2016-01-23 23:05:59,NaN,NaN,NaN,23,01,138,1afd7afbc81ecc1b13886a569d869e8a,4619,46
555120,91690261186ae5bee8f83808ea1e4a01,2016-01-23 17:27:23,NaN,NaN,NaN,23,01,104,91690261186ae5bee8f83808ea1e4a01,2218,20
555121,91690261186ae5bee8f83808ea1e4a01,2016-01-23 21:28:30,NaN,NaN,NaN,23,01,128,91690261186ae5bee8f83808ea1e4a01,2218,20
555122,d4ec2125aff74eded207d2d915ef682f,2016-01-23 11:03:07,NaN,NaN,NaN,23,01,66,d4ec2125aff74eded207d2d915ef682f,31088,51


In [18]:
# delete the 'region_hash' column because it is the same as 'start_region_hash'
del Supply_Demand_data['region_hash']
Supply_Demand_data.tail()

,start_region_hash,Time,Weather,temperature,PM2_5,Day,Month,Time_slot,Facilities,region_id
555118,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-23 13:15:33,NaN,NaN,NaN,23,01,79,6055,1
555119,1afd7afbc81ecc1b13886a569d869e8a,2016-01-23 23:05:59,NaN,NaN,NaN,23,01,138,4619,46
555120,91690261186ae5bee8f83808ea1e4a01,2016-01-23 17:27:23,NaN,NaN,NaN,23,01,104,2218,20
555121,91690261186ae5bee8f83808ea1e4a01,2016-01-23 21:28:30,NaN,NaN,NaN,23,01,128,2218,20
555122,d4ec2125aff74eded207d2d915ef682f,2016-01-23 11:03:07,NaN,NaN,NaN,23,01,66,31088,51


In [19]:
date = []
for ind in range(Supply_Demand_data.shape[0]):
    date.append(Supply_Demand_data['Time'][ind][:10])

del Supply_Demand_data['Time']
Supply_Demand_data = Supply_Demand_data.assign(Date = date)
Supply_Demand_data.tail()

,start_region_hash,Weather,temperature,PM2_5,Day,Month,Time_slot,Facilities,region_id,Date
555118,90c5a34f06ac86aee0fd70e2adce7d8a,NaN,NaN,NaN,23,01,79,6055,1,2016-01-23
555119,1afd7afbc81ecc1b13886a569d869e8a,NaN,NaN,NaN,23,01,138,4619,46,2016-01-23
555120,91690261186ae5bee8f83808ea1e4a01,NaN,NaN,NaN,23,01,104,2218,20,2016-01-23
555121,91690261186ae5bee8f83808ea1e4a01,NaN,NaN,NaN,23,01,128,2218,20,2016-01-23
555122,d4ec2125aff74eded207d2d915ef682f,NaN,NaN,NaN,23,01,66,31088,51,2016-01-23


# Handling NaN-Values

In [20]:
# Returns the most recurring number in the array
def mode(A):
    if len(A) == 0:
        return 'Error! List is empty.'
    else:
        Dict = {}
        A = np.array(A)
        for a in np.unique(A):
            Dict[str(a)] = (a!=0) * (A[A==a].shape[0])
        return max(Dict.iterkeys(), key=lambda k: Dict[k])

print mode([])
print int(float(mode([0.0, 0.0, 0.0])))
print mode([3.14, 2.76, 1.4, 2.0, 3.14, 0.0, 0, 0, 0.0, 0, 0, 0, 0])

Error! List is empty.
0
3.14


In [21]:
# Replace all the 'NaN' values in columns ['Weather', 'tempreture', 'PM2_5'] by value=0
Supply_Demand_data['Weather'] = Supply_Demand_data['Weather'].fillna(value=0)
Supply_Demand_data['temperature'] = Supply_Demand_data['temperature'].fillna(value=0)
Supply_Demand_data['PM2_5'] = Supply_Demand_data['PM2_5'].fillna(value=0)

In [22]:
# Declare dictionaries to store avg values of the columns for each time slot
mode_Weather = {}
mode_temperature = {}
mode_PM = {}

# Append the values at the time slot of each column respectively to their containers
for order in range(Supply_Demand_data.shape[0]):
    key = Supply_Demand_data['Date'][order] + str(Supply_Demand_data['Time_slot'][order])
    if key not in mode_Weather.keys():
        mode_Weather[key] = []
        mode_temperature[key] = []
        mode_PM[key] = []
    else:
        mode_Weather[key].append(Supply_Demand_data['Weather'][order])
        mode_temperature[key].append(Supply_Demand_data['temperature'][order])
        mode_PM[key].append(Supply_Demand_data['PM2_5'][order])
    
# Replace the containers with the most recurring value in them respectively
for key in mode_Weather.keys():
    mode_W = int(float(mode(mode_Weather[key])))
    if mode_W == 0:
        mode_Weather[key] = int(float(mode(Supply_Demand_data.Weather)))
    else:
        mode_Weather[key] = mode_W
        
    mode_t = int(float(mode(mode_temperature[key])))
    if mode_t == 0:
        mode_temperature[key] = int(float(mode(Supply_Demand_data.temperature)))
    else:
        mode_temperature[key] = mode_t

    mode_P = int(float(mode(mode_PM[key])))
    if mode_P == 0:
        mode_PM[key] = int(float(mode(Supply_Demand_data.PM2_5)))
    else:
        mode_PM[key] = mode_P

In [23]:
wther = []
tempr = []
pm2_5 = []
# Replace each zeros with the average values in the respective slot
for order in range(Supply_Demand_data.shape[0]):
    key = Supply_Demand_data['Date'][order] + str(Supply_Demand_data['Time_slot'][order])
    if Supply_Demand_data['Weather'][order] == 0:
        wther.append(mode_Weather[key])
    else:
        wther.append(Supply_Demand_data['Weather'][order])
    if Supply_Demand_data['temperature'][order] == 0:
        tempr.append(mode_temperature[key])
    else:
        tempr.append(Supply_Demand_data['temperature'][order])
    if Supply_Demand_data['PM2_5'][order] == 0:
        pm2_5.append(mode_Weather[key])
    else:
        pm2_5.append(Supply_Demand_data['PM2_5'][order])
        
Supply_Demand_data = Supply_Demand_data.assign(Weather = wther)
Supply_Demand_data = Supply_Demand_data.assign(temperature = tempr)
Supply_Demand_data = Supply_Demand_data.assign(PM2_5 = pm2_5)
Supply_Demand_data.tail()

,start_region_hash,Weather,temperature,PM2_5,Day,Month,Time_slot,Facilities,region_id,Date
555118,90c5a34f06ac86aee0fd70e2adce7d8a,3.0,-1.0,3.0,23,01,79,6055,1,2016-01-23
555119,1afd7afbc81ecc1b13886a569d869e8a,2.0,-4.0,2.0,23,01,138,4619,46,2016-01-23
555120,91690261186ae5bee8f83808ea1e4a01,4.0,4.0,4.0,23,01,104,2218,20,2016-01-23
555121,91690261186ae5bee8f83808ea1e4a01,4.0,4.0,4.0,23,01,128,2218,20,2016-01-23
555122,d4ec2125aff74eded207d2d915ef682f,4.0,4.0,4.0,23,01,66,31088,51,2016-01-23


In [26]:
Supply_Demand_data = Supply_Demand_data.drop_duplicates(subset = ['start_region_hash', 'Date', 'Time_slot'])

In [27]:
Supply_Demand_data.to_csv('Supply_Demand_test_data', index = False)